In [ ]:
#region Imports

# Import all the models, training functions, manipulators here

# Define the relative paths, append it to the system path
import sys
from pathlib import Path
project_root = Path.cwd().resolve().parents[2]
github_root = Path.cwd().resolve().parents[1]
sys.path.append(str(project_root))
sys.path.append(str(github_root))

print(f"Project root: {project_root}")
print(f"Github root: {github_root}")

# Imports related to models
from defs.helper_functions.training_functions import *
from defs.helper_functions.data_loaders import *
from defs.models.MLP import *
from defs.models.NIF import *
from defs.helper_functions.loaders import *
from defs.scripting.config_defs import config_from_args

# Time, to precisely: time
import time

# Garbage collector
import gc

# For export purposes
import pickle
import tensorboard as tb

# Metrics import
from sklearn.metrics import r2_score, mean_absolute_error
from torch.utils.flop_counter import FlopCounterMode
from defs.helper_functions.auxiliary import get_n_params

#endregion

Project root: C:\SenkDosya\Projects\AeroML
Github root: C:\SenkDosya\Projects\AeroML\initial-project


In [ ]:
#region Get Configs

cfg_run = config_from_args()

# Training config
cfg_train = cfg_run['cfg_train']

# Loader config
cfg_loader = cfg_train['cfg_loader']

# Model config
cfg_model_setup = cfg_run['cfg_model_setup']

# Data config
cfg_data = cfg_run['cfg_data']

dtype_pref = cfg_train['dtype']
if dtype_pref == 'float16': cfg_train['dtype'] = torch.float16
if dtype_pref == 'bfloat16': cfg_train['dtype'] = torch.bfloat16
if dtype_pref == 'float32': cfg_train['dtype'] = torch.float32
if dtype_pref == 'float64': cfg_train['dtype'] = torch.float64
else: raise ValueError(f"Unknown dtype: {dtype_pref} in cfg. Can be one of: 'float16', 'bfloat16, 'float32', 'float64'")

device_pref = cfg_train['device']
if device_pref == 'cuda' and torch.cuda.is_available(): device = torch.device('cuda'); cfg_train['device'] = device
if device_pref == 'cpu': device = torch.device('cpu'); cfg_train['device'] = device
else: raise ValueError(f"Unknown device: {device_pref} in cfg. Can be one of: 'cuda', 'cpu'")

#endregion

usage: ipykernel_launcher.py [-h] [--config CONFIG]
ipykernel_launcher.py: error: unrecognized arguments: --f="c:\Users\Ege Artan\AppData\Roaming\jupyter\runtime\kernel-v34640b73077ae10fe5824c5a17912ef968375dc45.json"


SystemExit: 2

c:\Users\Ege Artan\AppData\Local\Programs\Python\Python312\Lib\site-packages\IPython\core\interactiveshell.py:3707: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
#region Data Pre-Processing

df = pd.read_csv(rf"{str(project_root)}\{cfg_data['data_handle_from_project_root']}")

df = df.drop(['N1', 'N2'], axis=1) # Remove N1 and N2 since all the airfoils are subsonic

df['Reynolds'] = df['Reynolds'] / 1000000 # Normalize the Reynolds feature, too dominant

geom, cond, perf, names= get_dataset(df, 
                                    loc_geometry= cfg_data['loc_geometry'], 
                                    loc_cond= cfg_data['loc_cond'], 
                                    loc_perf_coeffs= cfg_data['loc_perf_coeffs'], 
                                    loc_names= cfg_data['loc_names']) # Get the necessary stuff for the dataset

ds = AirfoilDataset(geom, cond, perf, names)

del df, geom, cond, perf, names # Delete these to preserve memory

cfg_loader = {
    'n_epoch': 100,
    'n_train': 1000,
    'n_test': 17250,
    'train_batch': 1
}

dl_train, dl_val, dl_test = get_dataloaders(ds=ds, cfg_loader=cfg_loader)

dataloaders = [dl_train, dl_val, dl_test]; del dl_train, dl_val, dl_test

#endregion

In [ ]:
#region 